In [62]:
import xlwings as xw
import pandas as pd
import numpy as np
import re
from itertools import combinations

workbook_path = 'G:/B_Amex/Template.xlsm'
wb = xw.Book(workbook_path)
ws = wb.sheets['File Name']
ws1 = wb.sheets['Transaction Details 2']

def load_dataframe(worksheet, header_row=7):
    # Use 'A' + str(header_row) if your data starts from column A
    data_range = worksheet.range('A' + str(header_row)).expand('table')

    # Convert the data range to a DataFrame. By default, the header is taken from the first row of the range
    df = data_range.options(pd.DataFrame, index=False, header=True).value

    # If the headers are not properly captured, you might need to adjust the header argument
    return df

df_file_name = load_dataframe(ws)
df_transaction_details = load_dataframe(ws1)


In [63]:
def convert_dates(df, date_column):
    # Replace "Date Not Found" or any non-standard date strings with np.nan
    df[date_column] = df[date_column].astype(str).apply(
        lambda x: x if re.match(r'^\d{4}-\d{2}-\d{2}', x) else np.nan)

    # Now convert the cleaned date strings to datetime, errors='coerce' will handle any remaining non-date strings by converting them to NaT
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')

    return df

df_transaction_details = convert_dates(df_transaction_details, 'Date')
df_file_name = convert_dates(df_file_name, 'Date')

In [71]:
df_file_name

,File Name,File Path,Amount,Vendor,Date
0,Comcast_Business_Internet_Invoice_1766273.pdf,G:\t3nas\APPS\[02] Feb 2024\Comcast_Business_I...,1489.95,COMCAST,2024-02-14
1,Dell_Monitors_US_ORDERACK_f000c1d1-cf7f-49d4-a...,G:\t3nas\APPS\[02] Feb 2024\Dell_Monitors_US_O...,723.58,DELL,2024-02-14
2,Dell_Monitors_US_ORDERACK_f000c1d1-cf7f-49d4-a...,G:\t3nas\APPS\[02] Feb 2024\Dell_Monitors_US_O...,723.58,DELL,2024-02-14
3,Adobe Acrobat for Lauren.pdf,G:\t3nas\APPS\[02] Feb 2024\Adobe Acrobat for ...,27.54,ADOBE,2024-02-14
4,Server supply 4183063.pdf,G:\t3nas\APPS\[02] Feb 2024\Server supply 4183...,901.60,SERVER,2024-02-08
5,Apple_MacBook_Airs_AAA1533245 - Copy.pdf,G:\t3nas\APPS\[02] Feb 2024\Apple_MacBook_Airs...,4704.00,APPLE,2024-02-13
6,Apple_MacBook_Airs_AAA1533245.pdf,G:\t3nas\APPS\[02] Feb 2024\Apple_MacBook_Airs...,4704.00,APPLE,2024-02-13
7,Apple_MacBook_Pros_AAA1533249.pdf,G:\t3nas\APPS\[02] Feb 2024\Apple_MacBook_Pros...,3672.00,APPLE,2024-02-13
8,MSFT Project Plan 3 (Nonprofit Staff Pricing).pdf,G:\t3nas\APPS\[02] Feb 2024\MSFT Project Plan ...,24.00,MSFT,2024-02-12
9,MSFT Office 365 E3 (Nonprofit Staff Pricing).pdf,G:\t3nas\APPS\[02] Feb 2024\MSFT Office 365 E3...,1443.25,MSFT,2024-02-12


In [76]:
def find_matching_transactions(row, df_transaction_details, matched_transactions):
    vendor = row['Vendor']
    total = row['Amount']  # Assuming this column exists in df_file_name and represents the invoice total
    invoice_date = row['Date']  # Assuming 'Date' is already a datetime object
    file_name = row['File Name']

    # Filter transactions by the same vendor, date, and which have not been matched yet
    transactions = df_transaction_details[
        (df_transaction_details['Vendor'].str.contains(vendor, case=False, na=False)) &
        (df_transaction_details['Date'] == invoice_date) &
        (~df_transaction_details.index.isin(matched_transactions))
        ]

    # Strategy 1: Direct match between invoice total and transaction amount, matching dates
    direct_matches = transactions[transactions['Amount'] == total]
    if not direct_matches.empty:
        first_direct_match = direct_matches.index[0]  # get index of first direct match
        matched_transactions.update([first_direct_match])
        df_transaction_details.loc[first_direct_match, 'File name'] = file_name
        return
    
    # Strategy 2: Direct match between invoice total and transaction amount, excluding transactions with matching dates
    direct_matches_non_matching_date = df_transaction_details[
        (df_transaction_details['Vendor'].str.contains(vendor, case=False, na=False)) &
        (df_transaction_details['Amount'] == total) &
        (df_transaction_details['Date'] != invoice_date) &  # Exclude transactions with matching dates
        (~df_transaction_details.index.isin(matched_transactions))
    ]
    
    if not direct_matches_non_matching_date.empty:
        first_direct_match = direct_matches_non_matching_date.index[0]  # get index of first direct match
        matched_transactions.update([first_direct_match])
        df_transaction_details.loc[first_direct_match, 'Column1'] = 'Check Date'
        df_transaction_details.loc[first_direct_match, 'File name'] = file_name
        return

    # If no conditions are met, you might want to flag this for manual review
    print(f"No match found for {file_name} with Amount {total}")


matched_transactions = set()

# Iterate over the file name DataFrame and find matching transactions
df_transaction_details_copy = df_transaction_details.copy()
for index, row in df_file_name.iterrows():
    find_matching_transactions(row, df_transaction_details_copy, matched_transactions)

df_transaction_details_copy


No match found for Dell_Monitors_US_ORDERACK_f000c1d1-cf7f-49d4-ab05-11878fffa8f1 - Copy.pdf with Amount 723.58
No match found for Dell_Monitors_US_ORDERACK_f000c1d1-cf7f-49d4-ab05-11878fffa8f1.pdf with Amount 723.58
No match found for Apple_MacBook_Airs_AAA1533245 - Copy.pdf with Amount 4704.0
No match found for Apple_MacBook_Airs_AAA1533245.pdf with Amount 4704.0
No match found for Granite_1312024_03354959_635226634.pdf with Amount 43.25
No match found for calendly-billing_invoice_AzqVbSU5PZzivLBTV - Copy.pdf with Amount 864.0
No match found for calendly-billing_invoice_AzqVbSU5PZzivLBTV.pdf with Amount 864.0
No match found for Cloudflare_Feb_for_Marketing.pdf with Amount 250.0
No match found for Cloudflare_Feb_for_Comms.pdf with Amount 250.0


,Date,Receipt,Description,Amount,Account,Sub-Account,Vendor,Explantion,File name,Column1
0,2024-02-16,None,COMCAST BUSINESS PHILADELPHIA PA,1489.95,Please Review,00-0-005-0000-04,COMCAST,None,Comcast_Business_Internet_Invoice_1766273.pdf,Check Date
1,2024-02-15,None,DELL INC ROUND ROCK TX,321.59,Please Review,00-0-005-0000-04,DELL,None,None,None
2,2024-02-15,None,DELL INC ROUND ROCK TX,401.99,Please Review,00-0-005-0000-04,DELL,None,None,None
3,2024-02-14,None,ADOBE SYSTEMS Adobe SAN JOSE CA,27.54,5113.0,00-0-005-0000-04,ADOBE,None,Adobe Acrobat for Lauren.pdf,None
4,2024-02-14,None,AMAZON MARKEPLACE NA PA,14.84,5130.0,00-0-005-0000-04,AMAZON,None,Amazon_ESTATA_Data_Cables_ChicagoDR.pdf,None
5,2024-02-13,None,APPLE ONLINE STORE CUPERTINO CA,1568.00,5140.0,00-0-005-0000-04,APPLE,None,None,None
6,2024-02-13,None,APPLE ONLINE STORE CUPERTINO CA,3136.00,5140.0,00-0-005-0000-04,APPLE,None,None,None
7,2024-02-13,None,APPLE ONLINE STORE CUPERTINO CA,3672.00,5140.0,00-0-005-0000-04,APPLE,None,Apple_MacBook_Pros_AAA1533249.pdf,None
8,2024-02-13,None,MSFT *<E0500QZD1L> MSBILL.INFO,144.00,5113.0,00-0-005-0000-04,MSFT,None,MSFT Microsoft Defender for Office 365 (Plan 1...,Check Date
9,2024-02-13,None,MSFT *<E0500QZD1M> MSBILL.INFO,24.00,5113.0,00-0-005-0000-04,MSFT,None,MSFT Project Plan 3 (Nonprofit Staff Pricing).pdf,Check Date
